In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib.pyplot as plt
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake.all
    from pydrake.all import (
        RigidTransform, RotationMatrix, RollPitchYaw, RevoluteSpring, LogOutput, SignalLogger, ForceElement, 
        DoorHinge, DoorHingeConfig, PidController, FixedInputPortValue
    )

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

In [ ]:
# Constants
in2m = 0.0254

PAPER_WIDTH = 11*in2m
PAPER_DEPTH = 8.5*in2m

# Source:
# https://www.jampaper.com/paper-weight-chart.asp
PAPER_HEIGHT = 0.004*in2m
PAPER_DENSITY = 80/1000

# Sournce:
# https://smartech.gatech.edu/bitstream/handle/1853/5562/jones_ar.pdf
PAPER_YOUNGS_MODULUS = 10*1e9 # Convert to n/m^2

# I = b*h^3/12
PAPER_I = PAPER_DEPTH*PAPER_HEIGHT**3/12
# Stiffness = 3*(youngs modulus)*I/(Length)
PAPER_STIFFNESS = 3*PAPER_YOUNGS_MODULUS*PAPER_I # Missing 1/L^3

PEDESTAL_WIDTH = 0.225
PEDESTAL_HEIGHT = 0.1

FINGER_RADIUS = 0.01

epsilon = 0.001
PAPER_HEIGHT=0.007

In [ ]:
PAPER_STIFFNESS/PAPER_WIDTH

In [ ]:
def AddPedestal(plant):
    mu = 5.0
    # Parse pedestal model
    parser = pydrake.multibody.parsing.Parser(plant)
    pedestal_instance = parser.AddModelFromFile("pedestal.sdf")
    
    pedestal_body = plant.GetBodyByName("pedestal_base", pedestal_instance)
    width=PEDESTAL_WIDTH
    depth=0.2
    height=0.1
    i=0
#     for x in [-width/2.0, width/2.0]:
#         for y in [-depth/2.0, depth/2.0]:
#             for z in [-height/2.0, height/2.0]:
#                 plant.RegisterCollisionGeometry(
#                     pedestal_body,
#                     RigidTransform([x, y, z]),
#                     pydrake.geometry.Sphere(radius=1e-7), f"_contact_sphere{i}",
#                     pydrake.multibody.plant.CoulombFriction(mu, mu))
#                 i += 1
    
    # Weld pedestal to world
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("pedestal_base", pedestal_instance),
        RigidTransform(RotationMatrix(), [0, 0, PEDESTAL_HEIGHT/2])
    )
    
    return pedestal_instance

In [ ]:
def AddPointFinger(plant, init_x=PEDESTAL_WIDTH*0.6, init_z=0):
    mu = 1.0
    radius = FINGER_RADIUS
    finger = plant.AddModelInstance("finger")

    # Add false body at the origin that the finger 
    false_body = plant.AddRigidBody("false_body", finger,
        pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))

    # Initialize finger body
    finger_body = plant.AddRigidBody("body", finger, 
        pydrake.multibody.tree.SpatialInertia(
            mass=1.0,
            p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))

    # Register geometry
    shape = pydrake.geometry.Sphere(radius)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    
    # Add control joins for x and z movement
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_x",
        plant.world_frame(),
        plant.GetFrameByName("false_body"), [1, 0, 0], -1, 1))
    plant.AddJointActuator("finger_x", finger_x)
    finger_x.set_default_translation(init_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint(
        "finger_z",
        plant.GetFrameByName("false_body"),
        plant.GetFrameByName("body"), [0, 0, 1], -1, 1))
    finger_z.set_default_translation(init_z)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

In [ ]:
class FingerControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]
        output.SetFromVector(-finger_mass*g + [0, 0.05])

In [ ]:
class Paper:
    def __init__(self, plant, num_links, mu=5.0, width=PAPER_WIDTH, depth=PAPER_DEPTH, height=PAPER_HEIGHT, 
                 default_joint_angle=-np.pi/60):
        # Initialize parameters
        self.plant = plant
        self.num_links = num_links
        self.mu = mu
        self.width = width
        self.depth = depth
        self.height = height
        self.default_joint_angle = default_joint_angle
        
        self.link_width = self.width/self.num_links
        self.link_mass = PAPER_DENSITY*self.width*self.width/self.num_links
        self.stiffness = 0.00001
        self.damping = 0.00001
        self.name = "paper"
        
        self.paper_instances = []
        for link_num in range(self.num_links):
            # Initalize bodies and instances
            paper_instance = self.plant.AddModelInstance(self.name + str(link_num))
            paper_body = self.plant.AddRigidBody(
                self.name + "_body" + str(link_num),
                paper_instance,
                pydrake.multibody.tree.SpatialInertia(mass=self.link_mass, p_PScm_E=np.array([0., 0., 0.]),
                G_SP_E=pydrake.multibody.tree.UnitInertia.SolidBox(self.link_width, self.width, self.height))
            )

            # Set up collision
            if self.plant.geometry_source_is_registered():
                self.plant.RegisterCollisionGeometry(
                    paper_body,
                    RigidTransform(),
                    pydrake.geometry.Box(self.link_width, self.width, self.height),
                    self.name + "_body" + str(link_num), pydrake.multibody.plant.CoulombFriction(self.mu, self.mu)
                )
#                 i=0
#                 for x in [-self.link_width/2.0, self.link_width/2.0]:
#                     for y in [-self.width/2.0, self.width/2.0]:
#                         for z in [-self.height/2.0, self.height/2.0]:
#                             self.plant.RegisterCollisionGeometry(
#                                 paper_body,
#                                 RigidTransform([x, y, z]),
#                                 pydrake.geometry.Sphere(radius=1e-7), f"contact_sphere{i}_{link_num}",
#                                 pydrake.multibody.plant.CoulombFriction(self.mu, self.mu))
#                             i += 1
                self.plant.RegisterVisualGeometry(
                    paper_body,
                    RigidTransform(),
                    pydrake.geometry.Box(self.link_width, self.width, self.height),
                    self.name + "_body" + str(link_num),
                    [0.9, 0.9, 0.9, 1.0])

            # Set up joint actuators
            if link_num > 0:
                paper1_hinge_frame = pydrake.multibody.tree.FixedOffsetFrame(
                    "paper_hinge_frame",
                    self.plant.GetBodyByName("paper_body{}".format(link_num-1), self.paper_instances[-1]),
                    RigidTransform(RotationMatrix(), [self.link_width/2+epsilon/2, 0, 0.5*self.height]))
                self.plant.AddFrame(paper1_hinge_frame)
                paper2_hinge_frame = pydrake.multibody.tree.FixedOffsetFrame(
                    "paper_hinge_frame",
                    self.plant.GetBodyByName("paper_body{}".format(link_num), paper_instance),
                    RigidTransform(RotationMatrix(), [(-self.link_width/2+epsilon/2), 0, 0.5*self.height]))
                self.plant.AddFrame(paper2_hinge_frame)

                joint = self.plant.AddJoint(pydrake.multibody.tree.RevoluteJoint(
                    "paper_hinge",
                    paper1_hinge_frame,
                    paper2_hinge_frame,
                    [0, 1, 0]))

                joint.set_default_angle(self.default_joint_angle)
                ja = self.plant.AddJointActuator("joint actuator", joint)

            self.paper_instances.append(paper_instance)
        
        # Fix paper to object
        plant.WeldFrames(
            plant.world_frame(),
            plant.GetBodyByName("paper_body0", self.paper_instances[0]).body_frame(),
            RigidTransform(RotationMatrix(), [-(PEDESTAL_WIDTH/2-PAPER_WIDTH/4), 0, PEDESTAL_HEIGHT+PAPER_HEIGHT/2])
        )

    def init_ctrlrs(self, builder):
        """Initialize controllers for paper joints"""
        self.paper_ctrlrs = []
        for paper_instance in self.paper_instances[1:]:
            # In an ideal world, this would be done with a custom force element.
            # But since this is in Python, we can't implement the C++ class.
            paper_ctrlr = PidController(kp=[[self.stiffness]], ki=[[0]], kd=[[self.damping]])
            builder.AddSystem(paper_ctrlr)
            self.paper_ctrlrs.append(paper_ctrlr)
            builder.Connect(paper_ctrlr.get_output_port(), self.plant.get_actuation_input_port(paper_instance))
            builder.Connect(self.plant.get_state_output_port(paper_instance), paper_ctrlr.get_input_port_estimated_state())
    
    def connect_ctrlrs(self, diagram, diagram_context):
        """Fix controller input to an angle of zero."""
        for paper_ctrlr in self.paper_ctrlrs:
            paper_ctrlr_context = diagram.GetMutableSubsystemContext(paper_ctrlr, diagram_context)
            paper_ctrlr.get_input_port_desired_state().FixValue(paper_ctrlr_context, [0, 0])

In [ ]:
## Pre-finalize steps
builder = pydrake.systems.framework.DiagramBuilder()

num_links=10
L = PAPER_WIDTH/num_links
stiffness_N_p_m = PAPER_STIFFNESS/L**3
stiffness_N_p_rad = stiffness_N_p_m*L**2

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
pedestal_instance = AddPedestal(plant)
# paper_instances = AddPaper(plant, num_links=10, default_joint_angle=-0.001)
paper = Paper(plant, 10)
finger_instance = AddPointFinger(plant)
finger_ctrlr = FingerControl(plant)


stiffness_N_p_rad

In [ ]:
plant.Finalize()

## Post finalize steps
paper.init_ctrlrs(builder)

# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))

# Visualization and logging
poses_logger = LogOutput(plant.get_state_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
paper.connect_ctrlrs(diagram, diagram_context)

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
simulator.AdvanceTo(5)
vis.stop_recording()
vis.publish_recording()

In [ ]:
%matplotlib notebook

plt.plot(poses_logger.sample_times(), poses_logger.data()[0,:], label="Position")
plt.plot(poses_logger.sample_times(), poses_logger.data()[1,:], label="Velocity")
plt.axhline(-np.pi/2, linestyle='--', color='k', label='Setpoint')
plt.legend()
plt.xlabel("Time (s)")
plt.show()